<a href="https://colab.research.google.com/github/Ntorres95/nfl_big_data_bowl_2021/blob/main/passResult_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install fsspec
!pip install s3fs

     |████████████████████████████████| 133kB 8.7MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 6.9MB 9.6MB/s 
     |████████████████████████████████| 143kB 48.4MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 92kB 5.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 6.5MB 16.2MB/s 
     |████████████████████████████████| 1.3MB 42.5MB/s 
     |████████████████████████████████| 133kB 54.3MB/s 
     |████████████████████████████████| 552kB 55.4MB/s 
     |████████████████████████████████| 296kB 43.8MB/s 
     |██████████████████

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import boto3
import botocore
import os

In [ ]:
from boto3.session import Session
from botocore.client import Config
from botocore.handlers import set_list_objects_encoding_type_url

In [ ]:
weeks_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/weeksNFL.csv")
games_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/games.csv")
players_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/players.csv")
plays_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/plays.csv")

In [ ]:
weeks_df.shape

(18309388, 19)

In [ ]:
weeks_df = weeks_df.dropna()
games_df = games_df.dropna()
players_df = players_df.dropna()
plays_df = plays_df.dropna()

In [ ]:
weeks_df.shape

(5117955, 19)

In [ ]:
weeks_df.head(3)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH
10,2018-09-07T01:07:14.599Z,91.29,21.83,0.01,0.02,0.01,250.34,151.83,None,2555415.0,Austin Hooper,81.0,TE,1,away,2018090600,75,left,OUT


In [ ]:
games_df.head(3)

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,2018090600,09/06/2018,20:20:00,PHI,ATL,1
1,2018090901,09/09/2018,13:00:00,CLE,PIT,1
2,2018090902,09/09/2018,13:00:00,IND,CIN,1


In [ ]:
players_df.head(3)

,nflId,height,weight,birthDate,collegeName,position,displayName
0,2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
1,2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2,2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen


In [ ]:
plays_df.head(3)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
29,2018090600,1680,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.299050,False
45,2018090600,2503,(11:04) (Shotgun) M.Ryan pass incomplete short...,3,2,10,ATL,play_type_pass,ATL,32,EMPTY,"1 RB, 1 TE, 3 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,6.0,3.0,11:04:00,42.0,ING,ATL 2,I,0,-12,-1.868122,False
91,2018090901,439,(8:05) (Shotgun) B.Roethlisberger pass short r...,1,3,9,PIT,play_type_pass,PIT,43,EMPTY,"1 RB, 1 TE, 3 WR",6.0,4.0,"2 DL, 4 LB, 5 DB",TRADITIONAL,0.0,0.0,08:05:00,53.0,IBW,PIT 11,C,0,0,-1.770704,False


In [ ]:
df = weeks_df.merge(plays_df, how='right', on='playId')
df.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId_x,playId,playDirection,route,gameId_y,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018-09-07T02:17:33.400Z,110.35,32.48,0.00,0.00,0.00,44.48,307.08,None,2506467.0,Darren Sproles,43.0,RB,1.0,home,2.018091e+09,1680,left,WHEEL,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
1,2018-09-07T02:17:33.400Z,106.49,42.95,0.00,0.00,0.00,247.94,189.89,None,2507763.0,Mike Wallace,14.0,WR,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
2,2018-09-07T02:17:33.400Z,105.99,22.35,0.00,0.00,0.00,255.39,246.65,None,2540158.0,Zach Ertz,86.0,TE,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
3,2018-09-07T02:17:33.400Z,107.67,16.05,5.83,2.36,0.59,327.16,2.02,None,2552600.0,Nelson Agholor,13.0,WR,1.0,home,2.018091e+09,1680,left,CROSS,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
4,2018-09-07T02:17:33.400Z,106.75,15.20,0.00,0.00,0.00,253.47,288.27,None,2553502.0,DeAndre Carter,16.0,WR,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False


In [ ]:
df = df.drop(columns=['time', 'x', 'y', 's', 'a', 'o', 'dir', 'event', 'nflId', 'displayName', 'jerseyNumber', 'frameId', 'gameId_x', 'gameId_y', 'playDescription','possessionTeam', 'playType', 'yardlineSide', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers','typeDropback'])

In [ ]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

df = pd.get_dummies(df)

df.head()

,dis,playId,quarter,down,yardsToGo,yardlineNumber,defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore,absoluteYardlineNumber,offensePlayResult,playResult,epa,isDefensivePI,position_FB,position_HB,position_QB,position_RB,position_TE,position_WR,team_away,team_home,playDirection_left,playDirection_right,route_ANGLE,route_CORNER,route_CROSS,route_FLAT,route_GO,route_HITCH,route_IN,route_OUT,route_POST,route_SCREEN,route_SLANT,route_WHEEL,route_undefined,offenseFormation_EMPTY,offenseFormation_I_FORM,...,offenseFormation_WILDCAT,"personnelO_0 RB, 0 TE, 5 WR","personnelO_0 RB, 1 TE, 4 WR","personnelO_1 RB, 0 TE, 4 WR","personnelO_1 RB, 1 TE, 3 WR","personnelO_1 RB, 2 TE, 2 WR","personnelO_1 RB, 3 TE, 1 WR","personnelO_2 QB, 1 RB, 1 TE, 2 WR","personnelO_2 RB, 0 TE, 3 WR","personnelO_2 RB, 1 TE, 2 WR","personnelO_2 RB, 2 TE, 1 WR","personnelO_3 RB, 1 TE, 1 WR","personnelO_6 OL, 1 RB, 1 TE, 2 WR","personnelO_6 OL, 1 RB, 2 TE, 1 WR","personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL","personnelO_6 OL, 2 RB, 2 TE, 0 WR","personnelD_0 DL, 4 LB, 7 DB","personnelD_0 DL, 5 LB, 6 DB","personnelD_0 DL, 6 LB, 5 DB","personnelD_1 DL, 3 LB, 7 DB","personnelD_1 DL, 4 LB, 6 DB","personnelD_1 DL, 5 LB, 5 DB","personnelD_2 DL, 2 LB, 7 DB","personnelD_2 DL, 3 LB, 6 DB","personnelD_2 DL, 4 LB, 5 DB","personnelD_3 DL, 1 LB, 7 DB","personnelD_3 DL, 2 LB, 6 DB","personnelD_3 DL, 3 LB, 5 DB","personnelD_3 DL, 4 LB, 4 DB","personnelD_3 DL, 5 LB, 3 DB","personnelD_4 DL, 1 LB, 6 DB","personnelD_4 DL, 2 LB, 5 DB","personnelD_4 DL, 3 LB, 4 DB","personnelD_4 DL, 4 LB, 3 DB","personnelD_4 DL, 5 LB, 2 DB","personnelD_5 DL, 2 LB, 4 DB",passResult_C,passResult_I,passResult_IN,passResult_S
0,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,0.59,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
# Set features. This will also be used as your x values.
selected_features = df[['quarter', 'down', 'yardsToGo', 'defendersInTheBox', 'numberOfPassRushers', 'isDefensivePI',
                        'offenseFormation_EMPTY',	'offenseFormation_I_FORM',	'offenseFormation_JUMBO',	'offenseFormation_PISTOL',	'offenseFormation_SHOTGUN',	'offenseFormation_SINGLEBACK',
                        'offenseFormation_WILDCAT',	'personnelO_0 RB, 0 TE, 5 WR',	'personnelO_0 RB, 1 TE, 4 WR',	'personnelO_1 RB, 0 TE, 4 WR',	'personnelO_1 RB, 1 TE, 3 WR',	'personnelO_1 RB, 2 TE, 2 WR',	
                        'personnelO_1 RB, 3 TE, 1 WR',	'personnelO_2 QB, 1 RB, 1 TE, 2 WR',	'personnelO_2 RB, 0 TE, 3 WR',	'personnelO_2 RB, 1 TE, 2 WR',	'personnelO_2 RB, 2 TE, 1 WR',
                        'personnelO_3 RB, 1 TE, 1 WR',	'personnelO_6 OL, 1 RB, 1 TE, 2 WR',	'personnelO_6 OL, 1 RB, 2 TE, 1 WR',	'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL',	'personnelO_6 OL, 2 RB, 2 TE, 0 WR',
                        'personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	'personnelD_1 DL, 4 LB, 6 DB',
                        'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB',	'personnelD_2 DL, 3 LB, 6 DB',	'personnelD_2 DL, 4 LB, 5 DB',	'personnelD_3 DL, 1 LB, 7 DB',
                        'personnelD_3 DL, 2 LB, 6 DB',	'personnelD_3 DL, 3 LB, 5 DB',	'personnelD_3 DL, 4 LB, 4 DB',	'personnelD_3 DL, 5 LB, 3 DB',	'personnelD_4 DL, 1 LB, 6 DB',
                        'personnelD_4 DL, 2 LB, 5 DB',	'personnelD_4 DL, 3 LB, 4 DB',	'personnelD_4 DL, 4 LB, 3 DB',	'personnelD_4 DL, 5 LB, 2 DB',	'personnelD_5 DL, 2 LB, 4 DB'	]]

In [ ]:
defensive_features = df[['personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	'personnelD_1 DL, 4 LB, 6 DB',
                        'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB',	'personnelD_2 DL, 3 LB, 6 DB',	'personnelD_2 DL, 4 LB, 5 DB',	'personnelD_3 DL, 1 LB, 7 DB',
                        'personnelD_3 DL, 2 LB, 6 DB',	'personnelD_3 DL, 3 LB, 5 DB',	'personnelD_3 DL, 4 LB, 4 DB',	'personnelD_3 DL, 5 LB, 3 DB',	'personnelD_4 DL, 1 LB, 6 DB',
                        'personnelD_4 DL, 2 LB, 5 DB',	'personnelD_4 DL, 3 LB, 4 DB',	'personnelD_4 DL, 4 LB, 3 DB',	'personnelD_4 DL, 5 LB, 2 DB',	'personnelD_5 DL, 2 LB, 4 DB']]

In [ ]:
offensive_features = df[['offenseFormation_EMPTY',	'offenseFormation_I_FORM',	'offenseFormation_JUMBO',	'offenseFormation_PISTOL',	'offenseFormation_SHOTGUN',	'offenseFormation_SINGLEBACK',
                        'offenseFormation_WILDCAT',	'personnelO_0 RB, 0 TE, 5 WR',	'personnelO_0 RB, 1 TE, 4 WR',	'personnelO_1 RB, 0 TE, 4 WR',	'personnelO_1 RB, 1 TE, 3 WR',	'personnelO_1 RB, 2 TE, 2 WR',	
                        'personnelO_1 RB, 3 TE, 1 WR',	'personnelO_2 QB, 1 RB, 1 TE, 2 WR',	'personnelO_2 RB, 0 TE, 3 WR',	'personnelO_2 RB, 1 TE, 2 WR',	'personnelO_2 RB, 2 TE, 1 WR',
                        'personnelO_3 RB, 1 TE, 1 WR',	'personnelO_6 OL, 1 RB, 1 TE, 2 WR',	'personnelO_6 OL, 1 RB, 2 TE, 1 WR',	'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL',	'personnelO_6 OL, 2 RB, 2 TE, 0 WR',]]

**Catch Predictions**

Logistic Regression-----

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_C"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7214973850812001
Testing Data Score: 0.7213546346984276


Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9779619276148657

In [ ]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.22386823155737623, 'yardsToGo'),
 (0.13015459438402258, 'quarter'),
 (0.11573350923600438, 'defendersInTheBox'),
 (0.10372927948071403, 'numberOfPassRushers'),
 (0.08682015401164887, 'down'),
 (0.02765323164731775, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.02723720447568033, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.02231384081852296, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.021172215019413698, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.01981851446055935, 'offenseFormation_SHOTGUN'),
 (0.019152281816902678, 'isDefensivePI'),
 (0.01814854525007358, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.017157836270612598, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.015396007412204456, 'offenseFormation_SINGLEBACK'),
 (0.014855904597899306, 'offenseFormation_EMPTY'),
 (0.014786055407414145, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.0141295276736151, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.012351297811768119, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.010286610617093439, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.008340407134745504, 'personnelO_2 RB, 1 TE, 2

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] C=0.001, class_weight=None, max_iter=10, penalty=l1 .............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  C=0.001, class_weight=None, max_iter=10, penalty=l1, score=0.703, total=   1.7s
[CV] C=0.001, class_weight=None, max_iter=10, penalty=l1 .............


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.0s remaining:    0.0s


[CV]  C=0.001, class_weight=None, max_iter=10, penalty=l1, score=0.690, total=   3.3s
[CV] C=0.001, class_weight=None, max_iter=10, penalty=l1 .............


...

[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=l1, score=0.691, total=   3.2s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=l1 ..........


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=l1, score=0.689, total=   6.6s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=l1 ..........


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: penalty='none' is not supported for the liblinear solver

  FitFailedWarning)


[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=l1, score=0.693, total=   6.4s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=none ........
[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=none, score=nan, total=   0.1s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=none ........
[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=none, score=nan, total=   0.1s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=none ........
[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=none, score=nan, total=   0.1s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=none ........
[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=none, score=nan, total=   0.1s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=none ........
[CV]  C=0.01, class_weight=balanced, max_iter=10, penalty=none, score=nan, total=   0.1s
[CV] C=0.01, class_weight=balanced, max_iter=10, penalty=l2 ..........
[CV]  C=0.01, class_weight=balanced, max

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Incomplete Predictions**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_I"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Sack Predictions**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_S"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Clasifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Interceptions Predictions**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_IN"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Cacth Predictions vs Defensive formations**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_C"]
X = defensive_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, defensive_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Incomplete Passes vs Offense formations**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_I"]
X = offensive_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, offensive_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Exta DB's and Incomplete Catches**

Logistic Regression

In [ ]:
moreDB_features = df[['personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	
                      'personnelD_1 DL, 4 LB, 6 DB', 'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB', 'personnelD_2 DL, 4 LB, 5 DB',	
                      'personnelD_3 DL, 1 LB, 7 DB', 'personnelD_3 DL, 2 LB, 6 DB', 'personnelD_4 DL, 1 LB, 6 DB']]

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_I"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Extra DB and Interceptions**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_IN"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)



In [ ]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:

print(grid.best_params_)
print(grid.best_score_)

**Extra DB and Sacks**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_IN"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)


In [ ]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Extra DB's and Catches**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["passResult_C"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

**Offensive Formation agaisnt Extra DB's**

Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
y = df["offenseFormation_I_FORM"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

In [ ]:
# How can we test this againt catch or incomplete
# What does the test results actually tell us
# How can we improve this

In [ ]:
# show different style
# explain what we are trying to solve
# predicting y solves x
# Random Forest - these featurs were the most important
# grid search - 
# graphs of important features

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

Grid Search

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [.001, .01, .1, 1, 5, 10],
              'max_iter': [10, 100,1000],
              'class_weight': [None, 'balanced'],
              'penalty': ["l1","none", "l2"]}
model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

## **Kmeans**

# MatPlotLib